# Правило Ренча - пример [аллометрии](https://ru.wikipedia.org/wiki/Аллометрия)

[Правило Ренча](https://ru.wikipedia.org/wiki/%D0%9F%D1%80%D0%B0%D0%B2%D0%B8%D0%BB%D0%BE_%D0%A0%D0%B5%D0%BD%D1%87%D0%B0) (Rensch’s Rule)(положительная аллометрия полового диморфизма в размере) 
не подтверждается на черепахах.

[Koy W. Regis, Jesse M. Meik (2017) Allometry of sexual size dimorphism in turtles: a comparison of mass and length data
Ecology
Evolutionary Studies
Zoology
](https://peerj.com/articles/2914)

В 1950 году Rensch впервые описал, что в группах родственных видов половой диморфизм в размерах тела более выражен у крупных видов.

Правило Ренча не соблюдается у сов
[<cite data-cite="Abouheif1997">Abouheif E., Fairbairn D. J. A comparative analysis of allometry for sexual size dimorphism: assessing Rensch's rule //The American Naturalist. – 1997. – Т. 149. – №. 3. – С. 540-562.</cite>](http://www.journals.uchicago.edu/doi/abs/10.1086/286004)


Если поделить на группы, то правило выражено только у многоженцев 

![](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2211517/bin/rspb20071043f04.jpg)

[Интерактивный подробный просмотр](https://www.ncbi.nlm.nih.gov/core/lw/2.0/html/tileshop_pmc/tileshop_pmc_inline.html?title=Click%20on%20image%20to%20zoom&p=PMC3&id=2291161_rspb20071043f04.jpg)

Standardized size dimorphism (log male wing length−log female wing length) versus male size in subfamilies with different mating systems. Polygyny is associated with positive allometry (i.e. slopes >0 in these kinds of plots), and polyandry is associated with negative allometry (i.e. slopes <0). 

[Dale J. et al. Sexual selection explains Rensch's rule of allometry for sexual size dimorphism //Proceedings of the Royal Society of London B: Biological Sciences. – 2007. – Т. 274. – №. 1628. – С. 2971-2979.](https://www.ncbi.nlm.nih.gov/pubmed/17878139)

In [ ]:
%pylab inline
import pandas as pd
import seaborn as sns

In [ ]:
D = pd.read_excel('Data_S1_Complete_Dataset.xlsx', skiprows=5)
D=D.iloc[:,6:-5]
D.info()

## Первичное знакомство

Для первичного знакомства с данными построим скаттерограмму размеров мужских и женских карапаксов для одного из семейств - Сухопутные черепахи.

In [ ]:
D1 = D[(D.Family=='Testudinidae')]
D1.plot('M SCL (mm)','F SCL (mm)', kind='scatter', marker='o');
title('Размеры сухопутных черепах');

Видно, что крупные виды c панцирем более полуметра сильно выпадают из группы. Чтобы смягчить различия зададим логарифмическую шкалу.

In [ ]:
D1.plot('M SCL (mm)','F SCL (mm)', kind='scatter', marker='o', loglog=True);
title('Размеры сухопутных черепах в логарифмической шкале');

Из статьи про птиц следует, что логарифмирование является стандартным приемом при построении аллометрических функций.

## Расчет SSD

$$  SSD = log(X_{male}) - log(X_{female})$$

Согласно правилу Ренча sexual size dimorphism (SSD) зависит от общего размера тела в группе близкородственных видов. (Если самец крупнее - SSD растет, и наоборот).

Рассчитаем диморфизм для длины карапакса и для массы.

In [ ]:
D['SSD'] = log(D['M SCL (mm)']) - log(D['F SCL (mm)'])
D['SSD_mass'] = log(D['M Mass (g)']) - log(D['F Mass (g)'])
D.hist(['SSD','SSD_mass']);

Создадим колонки с логарифмированными значениями для задания оси абсцисс в будущем рисунке.

In [ ]:
D['log_size']=log(D['M SCL (mm)'])
D['log_mass']=log(D['M Mass (g)'])

Построим линейные зависимости по семействам. Посчитаем, сколько видов с данными есть по каждому семейству.

In [ ]:
kk=['M SCL (mm)','F SCL (mm)','M Mass (g)','F Mass (g)']
D.groupby('Family')[kk].count()

Оставим только те семейства, в которых больше 5 видов.

In [ ]:
popular_family = _[_['M Mass (g)']>5].index
popular_family

Отберем выборку представителей только этих семейств. Будьте внимательны, если создаете переменные, отличающиеся всего на один символ (тут добавлен знак подчеркивания).

In [ ]:
D_ = D[D.Family.isin(popular_family)]

In [ ]:
sns.lmplot('log_size', 'SSD', data=D_, hue='Family');


Как видно из рисунка в большинстве семейств правило Ренча соблюдается - линия тренда идет вверх при увеличении размера. У некоторых семейств (Podocnemididae) зависимость выражена слабо, и у одного (Cheloniidae) - слабо-отрицательная.

In [ ]:
sns.lmplot('log_mass', 'SSD_mass', data=D_, hue='Family');

Сходная картина с межполовыми различиями в массе.

Степень выраженности аллометрического правила оценивают по углу наклона линии тренда - или коэффициенту $a$ в уравнении $y= ax+b$.

## Перегруппировка

Попробуем проверить пару гипотез, связано ли соблюдение правило Ренча с образом жизни и с типом складывания шеи (подотряды).

Для этого нам надо создать новые колонки с факторами. 

In [ ]:
D.Family.unique()

Для пометки водные или не водные заведем указатель со всеми семействами. Значения фактора зададим:
 - m - морские
 - r - пресноводные (речные)
 - s - сухопутные.

In [ ]:
d = dict(zip(D.Family.unique(), 'mrsr' 'rmrs' 'srrm' 'rrr'))
d

In [ ]:
D['vita']= D.Family.apply(lambda s: d[s])
D['vita'].value_counts()

Подотряд можно задать сопоставлением каждого семейства со своим подотрядом, но поскольку у Черепах подотряда всего два, то можно проверить принадлежность к одному из двух семейств бокошейных, и если нет, то к скрытошейным.

In [ ]:
def podotryad(fam):
    subordo = 'Pleurodira' if fam in (
          'Podocnemididae',
          'Pelomedusidae', #Пеломедузовые
          'Chelidae'  #Змеиношейные
        ) else 'Cryptodira'
    
    return subordo

D['Subordo']=D.Family.apply(podotryad)
D['Subordo'].value_counts()

Повторим построение рисунков с новыми факторами.

In [ ]:
sns.lmplot('log_size', 'SSD', data=D, hue='Subordo');

In [ ]:
sns.lmplot('log_mass', 'SSD_mass', data=D, hue='Subordo');

In [ ]:
sns.lmplot('log_size', 'SSD', data=D, hue='vita');

In [ ]:
sns.lmplot('log_mass', 'SSD_mass', data=D, hue='vita');

После получения качественного решения, можно получить количественные значения коэффициентов подобранных моделей. Для этого для каждой группы нужно сделать анализ с помощью функции `polyfit` или пакета `statmodels`.